In [477]:
import cv2
import torch
import numpy as np

from ultralytics.nn.tasks import DetectionModel

## Chargement du modèle .best

In [ ]:
model_path = "/home/dim/clone_repo/BrickSearch/ouputs/output_models/lego_result/weights/best.pt"

In [ ]:
# Chargement du modèle sur le GPU 
checkpoint = torch.load(model_path, map_location=torch.device('cuda'))

model = checkpoint['model']
model.eval()

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(96, 192, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(192, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3k2(
      (cv1): Conv(
        (conv): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(192, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(384, 384, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(384, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
   

## Définition des classes et du choix de la classe

In [ ]:
classe_choisie = "1x2_Bleu"

In [ ]:
classes = [
    "1x2_Blanc",
    "1x2_Bleu",
    "1x2_Jaune",
    "1x2_Marron",
    "1x2_Noir",
    "1x2_Rouge",
    "1x2_Vert clear",
    "1x2_Vert dark",
    "1x4_Blanc",
    "1x4_Jaune",
    "1x4_Noir",
    "1x4_Rouge",
    "1x4_Vert clear",
    "1x4_Vert dark",
    "2x2_Blanc",
    "2x2_Bleu",
    "2x2_Jaune",
    "2x2_Marron",
    "2x2_Rouge",
    "2x2_Vert clear",
    "2x2_Vert dark",
    "2x4_Blanc",
    "2x4_Bleu",
    "2x4_Jaune",
    "2x4_Rouge",
    "2x4_Vert dark"
]

## Chargement et prétraitement de l'image

In [ ]:
image_path = "/home/dim/clone_repo/BrickSearch/data/dataset/test/images/IMG-20250325-WA0033.jpg"
image = cv2.imread(image_path)
if image is None:
    print("Erreur lors du chargement de l'image.")
    exit()

In [ ]:
# Conversion de BGR vers RGB et redimensionnement (ici à 640x640, idem à modele entrainné)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_resized = cv2.resize(image_rgb, (640, 640))

In [ ]:
# Conversion en tensor, normalisation, ajout de la dimension batch,
# transfert sur GPU et conversion en half precision
input_tensor = torch.from_numpy(image_resized).permute(2, 0, 1).float() / 255.0
input_tensor = input_tensor.unsqueeze(0)  # Ajout de la dimension batch
input_tensor = input_tensor.to(torch.device('cuda')).half()  # Passage en fp16

# Vérifier le type du tenseur avant l'inférence
print("Input tensor dtype:", input_tensor.dtype)  # Doit afficher torch.cuda.HalfTensor

Input tensor dtype: torch.float16


# Exécution de l'inférence

In [ ]:
with torch.no_grad():
    predictions = model(input_tensor)

## Annotation de l'image et enregistrement

In [ ]:
# Chargez l'image de test pour annotation
image_annotation = cv2.imread(image_path)
if image_annotation is None:
    raise ValueError("L'image n'a pas pu être chargée. Vérifiez le chemin.")

image_annotation = cv2.resize(image, (640, 640))

In [483]:
# Seuil de confiance pour considérer une prédiction comme valide
score_threshold = 0.82

In [484]:
# Réorganisation de la sortie :
# La prédiction initiale a la forme [1, 30, 8400]. On souhaite obtenir un tenseur de forme [8400, 30].
tensor_predictions = predictions[0]
preds = tensor_predictions.squeeze(0).transpose(0, 1) # shape : [8400, 30]

# predictions est un tenseur PyTorch, on le convertit en numpy pour faciliter le traitement avec OpenCV
preds = preds.cpu().detach().numpy()

In [ ]:
# Iterration sur les prédictions
for pred in preds: 
    # Extraction des coordonnées pour la boîte englobante
    x1, y1, x2, y2 = pred[:4]

    # récupèration du score maximum parmi les classes et son indice
    class_scores = pred[4:]
    max_class_score = np.max(class_scores)
    class_idx = np.argmax(class_scores)
    
    # On vérifie que le score de la classe prédite est supérieur au seuil et uniquement pour la classe cherchée
    if max_class_score > score_threshold and classe_choisie == classes[class_idx] :
        
        # On defini les point d'encadrement de la boîte englobante
            # x1, y1 : coordonnées du centre de la boîte
            # x2, y2 : dimensions de la boîte
        start_point = (int(x1-(x2/2)), int(y1-(y2/2)))
        end_point   = (int(x1+(x2/2)), int(y1+(y2/2)))
        
        # Dessine le rectangle 
        cv2.rectangle(image_annotation, start_point, end_point, color=(102, 178, 255), thickness=2)
        
        # Prépare le texte à annoter (nom de la classe et score)
        label = f"{classes[class_idx]}: {max_class_score:.2f}"
        
        # Place le texte au-dessus de la boîte
        cv2.putText(image_annotation, label, (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 
                    0.5, (102, 178, 255), thickness=1)

In [ ]:
# Enregistrement de l'image annotée
result_image_path = "/home/dim/clone_repo/BrickSearch/ouputs/images_annotees/image_annotée.jpg"
cv2.imwrite(result_image_path, image_annotation)

True